# 異步I/O進化之路
如今，地球上最發達、規模最龐大的計算機程序，莫過於因特網。而從CPU的時間觀中可知，網絡I/O是最大的I/O瓶頸，除了宕機沒有比它更慢的。所以，諸多異步框架都對準的是網絡I/O。

我們從一個爬蟲例子說起，從因特網上下載10篇網頁。

1. 阻塞

    * 程序未得到所需計算資源時被掛起的狀態。
    * 程序在等待某個操作完成期間，自身無法繼續幹別的事情，則稱該程序在該操作上是阻塞的。
    * 常見的阻塞形式有：網絡I/O阻塞、磁盤I/O阻塞、用戶輸入阻塞等。
    阻塞是無處不在的，包括CPU切換上下文時，所有的進程都無法真正幹事情，它們也會被阻塞。 （如果是多核CPU則正在執行上下文切換操作的核不可被利用。）

2. 非阻塞

    * 程序在等待某操作過程中，自身不被阻塞，可以繼續運行幹別的事情，則稱該程序在該操作上是非阻塞的。
    * 非阻塞並不是在任何程序級別、任何情況下都可以存在的。
    * 僅當程序封裝的級別可以囊括獨立的子程序單元時，它才可能存在非阻塞狀態。
    非阻塞的存在是因為阻塞存在，正因為某個操作阻塞導致的耗時與效率低下，我們才要把它變成非阻塞的。

3. 同步

    * 不同程序單元為了完成某個任務，在執行過程中需靠某種通信方式以協調一致，稱這些程序單元是同步執行的。
    例如購物系統中更新商品庫存，需要用“行鎖”作為通信信號，讓不同的更新請求強制排隊順序執行，那更新庫存的操作是同步的。
    簡言之，同步意味著有序。

4. 異步

    * 為完成某個任務，不同程序單元之間過程中無需通信協調，也能完成任務的方式。
    * 不相關的程序單元之間可以是異步的。
    例如，爬蟲下載網頁。調度程序調用下載程序後，即可調度其他任務，而無需與該下載任務保持通信以協調行為。不同網頁的下載、保存等操作都是無關的，也無需相互通知協調。這些異步操作的完成時刻並不確定。
    簡言之，異步意味著無序。
    上文提到的“通信方式”通常是指異步和並發編程提供的同步原語，如信號量、鎖、同步隊列等等。我們需知道，雖然這些通信方式是為了讓多個程序在一定條件下同步執行，但正因為是異步的存在，才需要這些通信方式。如果所有程序都是按序執行，其本身就是同步的，又何需這些同步信號呢？
    
5. 並發
    * 並發描述的是程序的組織結構。指程序要被設計成多個可獨立執行的子任務。
    * 以利用有限的計算機資源使多個任務可以被實時或近實時執行為目的。

6. 並行
    * 並行描述的是程序的執行狀態。指多個任務同時被執行。
    * 以利用富餘計算資源（多核CPU）加速完成多個任務為目的。
    並發提供了一種程序組織結構方式，讓問題的解決方案可以並行執行，但並行執行不是必須的。


## 同步且阻塞:

建立一個 blocking_way 的 http 請求，並且連發10個同步且阻塞的請求，計算時間！

In [6]:
import socket
from concurrent import futures


# 發起一個阻塞式的請求到 example.com
def blocking_way():
    sock = socket.socket()
    # blocking
    sock.connect(('example.com', 80))
    request = 'GET / HTTP/1.0\r\nHost: example.com\r\n\r\n'
    sock.send(request.encode('ascii'))
    response = b''
    chunk = sock.recv(4096)
    while chunk:
        response += chunk
        # blocking
        chunk = sock.recv(4096)
    return response


# 連發 10 個“同步且阻塞”請求
def sync_way():
    res = []
    for i in range(10):
        res.append(blocking_way())
    return len(res)


def main():
    start = time.time()
    print(sync_way(), '個http請求')
    print(time.time() - start, '秒')


if __name__ == '__main__':
    import time
    main()

10 個http請求
5.401376962661743 秒


如上圖所示，blocking_way() 的作用是建立 socket 連接，發送HTTP請求，然後從 socket 讀取HTTP響應並返回數據。示例中我們請求了 example.com 的首頁。在sync_way() 執行了10次，即下載 example.com 首頁10次。

在示例代碼中有兩個關鍵點。一是第10行的 sock.connect((‘example.com’, 80))，該調用的作用是向example.com主機的80端口發起網絡連接請求。二是第14行、第18行的sock.recv(4096)，該調用的作用是從socket上讀取4K字節數據。

我們知道，創建網絡連接，多久能創建完成不是客戶端決定的，而是由網絡狀況和服務端處理能力共同決定。服務端什麼時候返回了響應數據並被客戶端接收到可供程序讀取，也是不可預測的。所以sock.connect()和sock.recv()這兩個調用在默認情況下是阻塞的。

注：sock.send()函數並不會阻塞太久，它只負責將請求數據拷貝到TCP/IP協議棧的系統緩衝區中就返回，並不等待服務端返回的應答確認。

假設網絡環境很差，創建網絡連接需要1秒鐘，那麼sock.connect()就得阻塞1秒鐘，等待網絡連接成功。這1秒鐘對一顆2.6GHz的CPU來講，彷彿過去了83年，然而它不能幹任何事情。 sock.recv()也是一樣的必須得等到服務端的響應數據已經被客戶端接收。我們下載10篇網頁，這個阻塞過程就得重複10次。如果一個爬蟲系統每天要下載1000萬篇網頁呢？ ！

上面說了很多，我們力圖說明一件事：同步阻塞的網絡交互方式，效率低十分低下。特別是在網絡交互頻繁的程序中。這種方式根本不可能挑戰C10K/C10M。

## 改善的第一步：多進程 (阻塞但是非同步)

在一個程序內，依次執行10次太耗時，那開10個一樣的程序同時執行不就行了。於是我們想到了多進程編程。為什麼會先想到多進程呢？發展脈絡如此。在更早的操作系統（Linux 2.4）及其以前，進程是 OS 調度任務的實體，是面向進程設計的OS。

In [45]:
# 啟用多的進程來運行阻塞的http請求
def process_way():
    workers = 10
    with futures.ProcessPoolExecutor(workers) as executor:
        futs = {executor.submit(blocking_way) for i in range(10)}
    return len([fut.result() for fut in futs])


def main():
    start = time.time()
    print(process_way(), '個http請求')
    print(time.time() - start, '秒')


if __name__ == '__main__':
    import time
    main()

10 個http請求
0.49857616424560547 秒


改善效果立竿見影。但仍然有問題。總體耗時並沒有縮減到原來的十分之一，而是九分之一左右，還有一些時間耗到哪裡去了？進程切換開銷。

進程切換開銷不止像“CPU的時間觀”所列的“上下文切換”那麼低。 CPU從一個進程切換到另一個進程，需要把舊進程運行時的寄存器狀態、內存狀態全部保存好，再將另一個進程之前保存的數據恢復。對CPU來講，幾個小時就乾等著。當進程數量大於CPU核心數量時，進程切換是必然需要的。

除了切換開銷，多進程還有另外的缺點。一般的服務器在能夠穩定運行的前提下，可以同時處理的進程數在數十個到數百個規模。如果進程數量規模更大，系統運行將不穩定，而且可用內存資源往往也會不足。

多進程解決方案在面臨每天需要成百上千萬次下載任務的爬蟲系統，或者需要同時搞定數万並發的電商系統來說，並不適合。

除了切換開銷大，以及可支持的任務規模小之外，多進程還有其他缺點，如狀態共享等問題，後文會有提及，此處不再細究。

## 改進：多線程 (阻塞非同步)

因為要切換進程對 cpu 來說太久了，所以使用多線程再來執行，
雖然 python 有 GIL，不過來看看要多久時間。
由於線程的數據結構比進程更輕量級，同一個進程可以容納多個線程，從進程到線程的優化由此展開。後來的OS也把調度單位由進程轉為線程，進程只作為線程的容器，用於管理進程所需的資源。而且OS級別的線程是可以被分配到不同的CPU核心同時運行的。

In [46]:
# 啟動多個線程來發起阻塞式的 http 請求
def thread_way():
    workers = 10
    with futures.ThreadPoolExecutor(workers) as executor:
        futs = {executor.submit(blocking_way) for i in range(10)}
    return len([fut.result() for fut in futs])


def main():
    start = time.time()
    print(thread_way(), '個http請求')
    print(time.time() - start, '秒')


if __name__ == '__main__':
    import time
    main()

10 個http請求
0.4624929428100586 秒


結果符合預期，比多進程耗時要少些。從運行時間上看，多線程似乎已經解決了切換開銷大的問題。而且可支持的任務數量規模，也變成了數百個到數千個。

但是，多線程仍有問題，特別是Python裡的多線程。首先，Python中的多線程因為GIL的存在，它們並不能利用CPU多核優勢，一個Python進程中，只允許有一個線程處於運行狀態。那為什麼結果還是如預期，耗時縮減到了十分之一？

因為在做阻塞的系統調用時，例如sock.connect(),sock.recv()時，當前線程會釋放GIL，讓別的線程有執行機會。但是單個線程內，在阻塞調用上還是阻塞的。

小提示：Python中 time.sleep 是阻塞的，都知道使用它要謹慎，但在多線程編程中，time.sleep 並不會阻塞其他線程。

除了GIL之外，所有的多線程還有通病。它們是被OS調度，調度策略是搶占式的，以保證同等優先級的線程都有均等的執行機會，那帶來的問題是：並不知道下一時刻是哪個線程被運行，也不知道它正要執行的代碼是什麼。所以就可能存在競態條件。

例如爬蟲工作線程從任務隊列拿待抓取URL的時候，如果多個爬蟲線程同時來取，那這個任務到底該給誰？那就需要用到“鎖”或“同步隊列”來保證下載任務不會被重複執行。

而且線程支持的多任務規模，在數百到數千的數量規模。在大規模的高頻網絡交互系統中，仍然有些吃力。當然，多線程最主要的問題還是競態條件。

## 非組塞的開始：

我們利用 setblocking(False) 函數，來讓 socket 連線為非阻塞的形式！

## 同步非阻塞：

終於，我們來到了非阻塞解決方案。先來看看最原始的非阻塞如何工作的

In [47]:
import socket


# 建立一個非阻塞的 http 請求
def nonblocking_way():
    sock = socket.socket()
    # 設定阻塞為 False
    sock.setblocking(False)
    try:
        sock.connect(('example.com', 80))
    except BlockingIOError:
        # 非阻塞连接过程中也会抛出异常
        pass
    request = 'GET / HTTP/1.0\r\nHost: example.com\r\n\r\n'
    data = request.encode('ascii')
    # 不知道socket何时就绪，所以不断尝试发送
    while True:
        try:
            sock.send(data)
            # 直到send不抛异常，则发送完成
            break
        except OSError:
            pass

    response = b''
    while True:
        try:
            chunk = sock.recv(4096)
            while chunk:
                response += chunk
                chunk = sock.recv(4096)
            break
        except OSError:
            pass

    return response


# 以同步的方式發 10 個請求
def sync_way():
    res = []
    for i in range(10):
        res.append(nonblocking_way())
    return len(res)


def main():
    start = time.time()
    print(sync_way(), '個http請求')
    print(time.time() - start, '秒')


if __name__ == '__main__':
    import time
    main()

10 個http請求
5.577038049697876 秒


首先註意到兩點，就感覺被騙了。一是耗時與同步阻塞相當，二是代碼更複雜。要非阻塞何用？且慢。

上圖第9行代碼sock.setblocking(False)告訴OS，讓socket上阻塞調用都改為非阻塞的方式。之前我們說到，非阻塞就是在做一件事的時候，不阻礙調用它的程序做別的事情。上述代碼在執行完 sock.connect() 和 sock.recv() 後的確不再阻塞，可以繼續往下執行請求準備的代碼或者是執行下一次讀取。

代碼變得更複雜也是上述原因所致。第11行要放在try語句內，是因為socket在發送非阻塞連接請求過程中，系統底層也會拋出異常。 connect()被調用之後，立即可以往下執行第15和16行的代碼。

需要while循環不斷嘗試 send()，是因為connect()已經非阻塞，在send()之時並不知道 socket 的連接是否就緒，只有不斷嘗試，嘗試成功為止，即發送數據成功了。 recv()調用也是同理。

雖然connect() 和recv() 不再阻塞主程序，空出來的時間段CPU沒有空閒著，但並沒有利用好這空閒去做其他有意義的事情，而是在循環嘗試讀寫socket （不停判斷非阻塞調用的狀態是否就緒）。還得處理來自底層的可忽略的異常。也不能同時處理多個 socket 。

然後10次下載任務仍然按序進行。所以總體執行時間和同步阻塞相當。如果非得這樣子，那還不如同步阻塞算了。

## 非阻塞的改進：

1. epoll:
    判斷非阻塞調用是否就緒如果 OS 能做，是不是應用程序就可以不用自己去等待和判斷了，就可以利用這個空閒去做其他事情以提高效率。

    所以OS將I/O狀態的變化都封裝成了事件，如可讀事件、可寫事件。並且提供了專門的系統模塊讓應用程序可以接收事件通知。這個模塊就是select。讓應用程序可以通過select註冊文件描述符和回調函數。當文件描述符的狀態發生變化時，select 就調用事先註冊的回調函數。

    select因其算法效率比較低，後來改進成了poll，再後來又有進一步改進，BSD內核改進成了kqueue模塊，而Linux內核改進成了epoll模塊。這四個模塊的作用都相同，暴露給程序員使用的API也幾乎一致，區別在於kqueue 和 epoll 在處理大量文件描述符時效率更高。

    鑑於 Linux 服務器的普遍性，以及為了追求更高效率，所以我們常常聽聞被探討的模塊都是 epoll 。

2. callback:
    把I/O事件的等待和監聽任務交給了 OS，那 OS 在知道I/O狀態發生改變後（例如socket連接已建立成功可發送數據），它又怎麼知道接下來該干嘛呢？只能回調。

    需要我們將發送數據與讀取數據封裝成獨立的函數，讓epoll代替應用程序監聽socket狀態時，得告訴epoll：“如果socket狀態變為可以往裡寫數據（連接建立成功了），請調用HTTP請求發送函數。如果socket 變為可以讀數據了（客戶端已收到響應），請調用響應處理函數。
    
## 於是我們利用epoll結合回調機制重構爬蟲代碼：

In [22]:
import socket
from selectors import DefaultSelector, EVENT_WRITE, EVENT_READ

selector = DefaultSelector()
stopped = False
urls_todo = {'/', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9'}


class Crawler:
    def __init__(self, url):
        self.url = url
        self.sock = None
        self.response = b''

    def fetch(self):
        self.sock = socket.socket()
        self.sock.setblocking(False)
        try:
            self.sock.connect(('example.com', 80))
        except BlockingIOError:
            pass
        selector.register(self.sock.fileno(), EVENT_WRITE, self.connected)

    def connected(self, key, mask):
        selector.unregister(key.fd)
        get = 'GET {0} HTTP/1.0\r\nHost: example.com\r\n\r\n'.format(self.url)
        self.sock.send(get.encode('ascii'))
        # 註冊到 selector
        selector.register(key.fd, EVENT_READ, self.read_response)

    def read_response(self, key, mask):
        global stopped
        # 如果响应大于4KB，下一次循环会继续读
        chunk = self.sock.recv(4096)
        if chunk:
            self.response += chunk
        else:
            selector.unregister(key.fd)
            urls_todo.remove(self.url)
            if not urls_todo:
                stopped = True

此處和前面稍有不同的是，我們將下載不同的10個頁面，相對URL路徑存放於urls_todo集合中。現在看看改進在哪。

首先，不斷嘗試send() 和 recv() 的兩個循環被消滅掉了。

其次，導入了selectors模塊，並創建了一個DefaultSelector 實例。 Python標準庫提供的selectors模塊是對底層select/poll/epoll/kqueue的封裝。 DefaultSelector類會根據 OS 環境自動選擇最佳的模塊，那在 Linux 2.5.44 及更新的版本上都是epoll了。

然後，在第25行和第31行分別註冊了socket可寫事件(EVENT_WRITE)和可讀事件(EVENT_READ)發生後應該採取的回調函數。

雖然代碼結構清晰了，阻塞操作也交給OS去等待和通知了，但是，我們要抓取10個不同頁面，就得創建10個Crawler實例，就有20個事件將要發生，那如何從selector裡獲取當前正發生的事件，並且得到對應的回調函數去執行呢？

## Event loop

為了解決上述問題，那我們只得採用老辦法，寫一個循環，去訪問selector模塊，

等待它告訴我們當前是哪個事件發生了，應該對應哪個回調。這個等待事件通知的循環，稱之為事件循環。


In [23]:
def loop():
    while not stopped:
        # 阻塞, 直到一个事件发生
        events = selector.select()
        for event_key, event_mask in events:
            callback = event_key.data
            callback(event_key, event_mask)

上述代碼中，我們用stopped全局變量控制事件循環何時停止。當urls_todo消耗完畢後，會標記stopped為True。

重要的是第49行代碼，selector.select() 是一個阻塞調用，因為如果事件不發生，那應用程序就沒事件可處理，所以就乾脆阻塞在這裡等待事件發生。那可以推斷，如果只下載一篇網頁，一定要connect()之後才能send()繼而recv()，那它的效率和阻塞的方式是一樣的。因為不在connect()/recv()上阻塞，也得在select()上阻塞。

所以，selector機制(後文以此稱呼代指epoll/kqueue)是設計用來解決大量並發連接的。當系統中有大量非阻塞調用，能隨時產生事件的時候，selector機制才能發揮最大的威力。

下面是如何啟創建10個下載任務和啟動事件循環的：

In [25]:
selector = DefaultSelector()
stopped = False
urls_todo = {'/', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9'}


class Crawler:
    def __init__(self, url):
        self.url = url
        self.sock = None
        self.response = b''

    def fetch(self):
        self.sock = socket.socket()
        self.sock.setblocking(False)
        try:
            self.sock.connect(('example.com', 80))
        except BlockingIOError:
            pass
        selector.register(self.sock.fileno(), EVENT_WRITE, self.connected)

    def connected(self, key, mask):
        selector.unregister(key.fd)
        get = 'GET {0} HTTP/1.0\r\nHost: example.com\r\n\r\n'.format(self.url)
        self.sock.send(get.encode('ascii'))
        # 註冊到 selector
        selector.register(key.fd, EVENT_READ, self.read_response)

    def read_response(self, key, mask):
        global stopped
        # 如果响应大于4KB，下一次循环会继续读
        chunk = self.sock.recv(4096)
        if chunk:
            self.response += chunk
        else:
            selector.unregister(key.fd)
            urls_todo.remove(self.url)
            if not urls_todo:
                stopped = True

def loop():
    while not stopped:
        # 阻塞, 直到一个事件发生
        events = selector.select()
        for event_key, event_mask in events:
            callback = event_key.data
            callback(event_key, event_mask)

if __name__ == '__main__':
    import time
    start = time.time()
    for url in urls_todo:
        crawler = Crawler(url)
        crawler.fetch()
    loop()
    print(time.time() - start)

0.4746260643005371


上述執行結果令人振奮。在單線程內用 事件循環+回調 搞定了10篇網頁同時下載的問題。這，已經是異步編程了。雖然有一個for 循環順序地創建Crawler 實例並調用 fetch 方法，但是fetch 內僅有connect()和註冊可寫事件，而且從執行時間明顯可以推斷，多個下載任務確實在同時進行！

上述代碼異步執行的過程：

1. 創建Crawler 實例；
2. 調用fetch方法，會創建socket連接和在selector上註冊可寫事件；
3. fetch內並無阻塞操作，該方法立即返回；
4. 重複上述3個步驟，將10個不同的下載任務都加入事件循環；
5. 啟動事件循環，進入第1輪循環，阻塞在事件監聽上；
6. 當某個下載任務EVENT_WRITE被觸發，回調其connected方法，第一輪事件循環結束；
7. 進入第2輪事件循環，當某個下載任務有事件觸發，執行其回調函數；此時已經不能推測是哪個事件發生，因為有可能是上次connected裡的EVENT_READ先被觸發，也可能是其他某個任務的EVENT_WRITE被觸發；（此時，原來在一個下載任務上會阻塞的那段時間被利用起來執行另一個下載任務了）
8. 循環往復，直至所有下載任務被處理完成
9. 退出事件循環，結束整個下載程序

## 結論

目前為止，我們已經從同步阻塞學習到了異步非阻塞。掌握了在單線程內同時並發執行多個網絡I/O阻塞型任務的黑魔法。而且與多線程相比，連線程切換都沒有了，執行回調函數是函數調用開銷，在線程的棧內完成，因此性能也更好，單機支持的任務規模也變成了數万到數十萬個。 （不過我們知道：沒有免費午餐，也沒有銀彈。）

部分編程語言中，對異步編程的支持就止步於此（不含語言官方之外的擴展）。需要程序猿直接使用epoll去註冊事件和回調、維護一個事件循環，然後大多數時間都花在設計回調函數上。

通過本節的學習，我們應該認識到，不論什麼編程語言，但凡要做異步編程，上述的“事件循環+回調”這種模式是逃不掉的，儘管它可能用的不是epoll，也可能不是while循環。如果你找到了一種不屬於 “等會兒告訴你” 模型的異步方式，請立即給我打電話（注意，打電話是Call）。

為什麼我們在某些​​異步編程中並沒有看到 CallBack 模式呢？這就是我們接下來要探討的問題。本節是學習異步編程的一個終點，也是另一個起點。畢竟咱們講 Python 異步編程，還沒提到其主角協程的用武之地。

# python 的異步優化

我們將在本節學習到 Python 生態對異步編程的支持是如何繼承前文所述的“事件循環+回調”模式演變到asyncio的原生協程模式。

## 回調地獄：

我們已經學會了“事件循環+回調”的基本運行原理，可以基於這種方式在單線程內實現異步編程。也確實能夠大大提高程序運行效率。但是，剛才所學的只是最基本的，然而在生產項目中，要應對的複雜度會大大增加。考慮如下問題：

1. 如果回調函數執行不正常該如何？
2. 如果回調裡面還要嵌套回調怎麼辦？要嵌套很多層怎麼辦？
3. 如果嵌套了多層，其中某個環節出錯了會造成什麼後果？
4. 如果有個數據需要被每個回調都處理怎麼辦？

In [27]:
# def callback_1():
    # # processing ...
    # def callback_2():
        # # processing.....
        # def callback_3():
            # # processing ....
            # def callback_4():
                # #processing .....
                # def callback_5():
                    # # processing ......
                # async_function(callback_5)
            # async_function(callback_4)
        # async_function(callback_3)
    # async_function(callback_2)
# async_function(callback_1)


def a(result):
    result = result + 'a'
    print(result)
    return result

def b(result):
    result = result + 'b'
    print(result)
    return result

def c(result):
    result = result + 'c'
    print(result)
    return result


a(b(c('1')))

1c
1cb
1cba


'1cba'

我上面實際也是回調地獄式的風格，但這不是主要矛盾。主要在於，原本從上而下的代碼結構，要改成從內到外的。先f，再e，再d，…，直到最外層 a 執行完成。在同步版本中，執行完a後執行b，這是線程的指令指針控制著的流程，而在回調版本中，流程就是程序猿需要注意和安排的。

* 共享狀態管理困難:

    回顧第3節爬蟲代碼，同步阻塞版的sock對像從頭使用到尾，而在回調的版本中，我們必須在Crawler實例化後的對象self裡保存它自己的sock對象。如果不是採用OOP的編程風格，那需要把要共享的狀態接力似的傳遞給每一個回調。多個異步調用之間，到底要共享哪些狀態，事先就得考慮清楚，精心設計。
    
* 錯誤處理困難:

    一連串的回調構成一個完整的調用鏈。例如上述的 a 到 f。假如 d 拋了異常怎麼辦？整個調用鏈斷掉，接力傳遞的狀態也會丟失，這種現象稱為調用棧撕裂。 c 不知道該干嘛，繼續異常，然後是 b 異常，接著 a 異常。好嘛，報錯日誌就告訴你，a 調用出錯了，但實際是 d 出錯。所以，為了防止棧撕裂，異常必須以數據的形式返回，而不是直接拋出異常，然後每個回調中需要檢查上次調用的返回值，以防錯誤吞沒。
    
如果說代碼風格難看是小事，但棧撕裂和狀態管理困難這兩個缺點會讓基於回調的異步編程很艱難。所以不同編程語言的生態都在致力於解決這個問題。才誕生了後來的Promise、Co-routine等解決方案。

Python 生態也以終為始，秉承著“程序猿不必難程序猿”的原則，讓語言和框架開發者苦逼一點，也要讓應用開發者舒坦。在事件循環+回調的基礎上衍生出了基於協程的解決方案，代表作有 Tornado、Twisted、asyncio 等。接下來我們隨著 Python 生態異步編程的發展過程，深入理解Python異步編程。


## 核心問題
通過前面的學習，我們清楚地認識到異步編程最大的困難：異步任務何時執行完畢？接下來要對異步調用的返回結果做什麼操作？

上述問題我們已經通過事件循環和回調解決了。但是回調會讓程序變得複雜。要異步，必回調，又是否有辦法規避其缺點呢？那需要弄清楚其本質，為什麼回調是必須的？還有使用回調時克服的那些缺點又是為了什麼？

答案是程序為了知道自己已經乾了什麼？正在幹什麼？將來要幹什麼？換言之，程序得知道當前所處的狀態，而且要將這個狀態在不同的回調之間延續下去。

多個回調之間的狀態管理困難，那讓每個回調都能管理自己的狀態怎麼樣？鍊式調用會有棧撕裂的困難，讓回調之間不再鍊式調用怎樣？不鍊式調用的話，那又如何讓被調用者知道已經完成了？那就讓這個回調通知那個回調如何？而且一個回調，不就是一個待處理任務嗎？

任務之間得相互通知，每個任務得有自己的狀態。那不就是很古老的編程技法：協作式多任務？然而要在單線程內做調度，啊哈，協程！每個協程具有自己的棧幀，當然能知道自己處於什麼狀態，協程之間可以協作那自然可以通知別的協程。

## 協程(co-routine)

早期的 Pythoner 發現 Python 中有種特殊的對象——生成器（Generator），它的特點和協程很像。每一次迭代之間，會暫停執行，繼續下一次迭代的時候還不會丟失先前的狀態。

為了支持用生成器做簡單的協程，Python 2.5 對生成器進行了增強(PEP 342)，該增強提案的標題是 “Coroutines via Enhanced Generators”。有了PEP 342的加持，生成器可以通過yield 暫停執行和向外返回數據，也可以通過send()向生成器內發送數據，還可以通過throw()向生成器內拋出異常以便隨時終止生成器的運行。

接下來，我們用基於生成器的協程來重構先前的爬蟲代碼。

## 未來對象(Future)

不用回調的方式了，怎麼知道異步調用的結果呢？先設計一個對象，異步調用執行完的時候，就把結果放在它裡面。這種對象稱之為未來對象。

In [28]:
class Future:
    def __init__(self):
        self.result = None
        self._callbacks = []

    def add_done_callback(self, fn):
        self._callbacks.append(fn)

    def set_result(self, result):
        self.result = result
        for fn in self._callbacks:
            fn(self)

未來對像有一個result屬性，用於存放未來的執行結果。還有個set_result()方法，是用於設置result的，並且會在給result綁定值以後運行事先給future添加的回調。回調是通過未來對象的add_done_callback()方法添加的。

不要疑惑此處的callback，說好了不回調的嘛？難道忘了我們曾經說的要異步，必回調。不過也別急，此處的回調，和先前學到的回調，還真有點不一樣。

## 重構 Crawler

現在不論如何，我們有了未來對象可以代表未來的值。先用Future來重構爬蟲代碼。

In [29]:
import socket
from selectors import DefaultSelector, EVENT_WRITE, EVENT_READ

selector = DefaultSelector()
stopped = False
urls_todo = {'/', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9'}


class Crawler:
    def __init__(self, url):
        self.url = url
        self.response = b''

    def fetch(self):
        sock = socket.socket()
        sock.setblocking(False)
        try:
            sock.connect(('example.com', 80))
        except BlockingIOError:
            pass
        f = Future()

        def on_connected():
            f.set_result(None)

        selector.register(sock.fileno(), EVENT_WRITE, on_connected)
        yield f
        selector.unregister(sock.fileno())
        get = 'GET {0} HTTP/1.0\r\nHost: example.com\r\n\r\n'.format(self.url)
        sock.send(get.encode('ascii'))

        global stopped
        while True:
            f = Future()

            def on_readable():
                f.set_result(sock.recv(4096))

            selector.register(sock.fileno(), EVENT_READ, on_readable)
            chunk = yield f
            selector.unregister(sock.fileno())
            if chunk:
                self.response += chunk
            else:
                urls_todo.remove(self.url)
                if not urls_todo:
                    stopped = True
                break

和先前的回調版本對比，已經有了較大差異。 fetch 方法內有了yield表達式，使它成為了生成器。我們知道生成器需要先調用next()迭代一次或者是先send(None)啟動，遇到yield之後便暫停。那這fetch生成器如何再次恢復執行呢？至少 Future 和 Crawler都沒看到相關代碼。

## 任務對象(Task)

為了解決上述問題，我們只需遵循一個編程規則：單一職責，每種角色各司其職，如果還有工作沒有角色來做，那就創建一個角色去做。沒人來恢復這個生成器的執行麼？沒人來管理生成器的狀態麼？創建一個，就叫Task好了，很合適的名字。

In [30]:
class Task:
    def __init__(self, coro):
        self.coro = coro
        f = Future()
        f.set_result(None)
        self.step(f)

    def step(self, future):
        try:
            # send会进入到coro执行, 即fetch, 直到下次yield
            # next_future 为yield返回的对象
            next_future = self.coro.send(future.result)
        except StopIteration:
            return
        next_future.add_done_callback(self.step)

上述代碼中Task封裝了coro對象，即初始化時傳遞給他的對象，被管理的任務是待執行的協程，故而這裡的coro就是fetch()生成器。它還有個step()方法，在初始化的時候就會執行一遍。 step()內會調用生成器的send()方法，初始化第一次發送的是None就驅動了coro即fetch()的第一次執行。

send()完成之後，得到下一次的future，然後給下一次的future添加step()回調。原來add_done_callback()不是給寫爬蟲業務邏輯用的。此前的callback可就乾的是業務邏輯呀。

再看fetch()生成器，其內部寫完了所有的業務邏輯，包括如何發送請求，如何讀取響應。而且註冊給selector的回調相當簡單，就是給對應的future對象綁定結果值。兩個yield表達式都是返回對應的future對象，然後返回Task.step()之內，這樣Task, Future, Coroutine三者精妙地串聯在了一起。

初始化Task對像以後，把fetch()給驅動到了第44行yied f就完事了，接下來怎麼繼續？

## 事件循環(Event Loop)來驅動

該事件循環上場了。接下來，只需等待已經註冊的EVENT_WRITE事件發生。事件循環就像心臟一般，只要它開始跳動，整個程序就會持續運行。

In [31]:
def loop():
    while not stopped:
        # 阻塞, 直到一个事件发生
        events = selector.select()
        for event_key, event_mask in events:
            callback = event_key.data
            callback()


if __name__ == '__main__':
    import time
    start = time.time()
    for url in urls_todo:
        crawler = Crawler(url)
        Task(crawler.fetch())
    loop()
    print(time.time() - start)


0.4742879867553711


現在loop有了些許變化，callback()不再傳遞event_key和event_mask參數。也就是說，這裡的回調根本不關心是誰觸發了這個事件，結合fetch()可以知道，它只需完成對future設置結果值即可f.set_result()。而且future是誰它也不關心，因為協程能夠保存自己的狀態，知道自己的future是哪個。也不用關心到底要設置什麼值，因為要設置什麼值也是協程內安排的。
而基於生成器協程的風格：

無鍊式調用
selector的回調裡只管給future設置值，不再關心業務邏輯
loop 內回調callback()不再關注是誰觸發了事件
已趨近於同步代碼的結構
無需程序員在多個協程之間維護狀態，例如哪個才是自己的sock
此時的loop()，真的成了一個心臟，它只管往外泵血，不論這份血液是要輸送給大腦還是要給腳趾，只要它還在跳動，生命就能延續。

## 生成器協程風格和回調風格對比總結

在回調風格中：

* 存在鍊式回調（雖然示例中嵌套回調只有一層）
* 請求和響應也不得不分為兩個回調以至於破壞了同步代碼那種結構
* 程序員必須在回調之間維護必須的狀態。
* 更多...

而基於生成器協程的風格：

* 無鍊式調用
* selector的回調裡只管給future設置值，不再關心業務邏輯
* loop 內回調callback()不再關注是誰觸發了事件
* 已趨近於同步代碼的結構
* 無需程序員在多個協程之間維護狀態，例如哪個才是自己的sock

## code 還是很醜，可以重構嗎？

如果說fetch的容錯能力要更強，業務功能也需要更完善，怎麼辦？而且技術處理的部分（socket相關的）和業務處理的部分（請求與返回數據的處理）混在一起。

* 創建socket連接可以抽象復用吧？
* 循環讀取整個response可以抽象復用吧？
* 循環內處理socket.recv()的可以抽象復用吧？
* 但是這些關鍵節點的地方都有yield，抽離出來的代碼也需要是生成器。而且fetch()自己也得是生成器。生成器裡玩生成器，代碼好像要寫得更醜才可以……

Python 語言的設計者們也認識到了這個問題，再次秉承著“程序猿不必為難程序猿”的原則，他們搗鼓出了一個yield from來解決生成器裡玩生成器的問題。

## 用 yield from 改進生成器協程

yield from 是Python 3.3 新引入的語法（PEP 380）。它主要解決的就是在生成器裡玩生成器不方便的問題。它有兩大主要功能。

第一個功能是：讓嵌套生成器不必通過循環迭代yield，而是直接yield from。以下兩種在生成器裡玩子生成器的方式是等價的。

In [34]:
def gen_one():
    subgen = range(10)
    yield from subgen
def gen_two():
    subgen = range(10)
    for item in subgen:        
        yield item

第二個功能就是在子生成器和原生成器的調用者之間打開雙向通道，兩者可以直接通信。

In [37]:
def gen():
    yield from subgen()
def subgen():
    while True:
        x = yield
        yield x+1
def main():
    g = gen()
    next(g)                # 驱动生成器g开始执行到第一个 yield
    retval = g.send(1)     # 看似向生成器 gen() 发送数据
    print(retval)          # 返回2
    g.throw(StopIteration) # 看似向gen()抛入异常

通過上述代碼清晰地理解了yield from的雙向通道功能。關鍵字yield from在gen()內部為subgen()和main()開闢了通信通道。 main()裡可以直接將數據1發送給subgen(),subgen()也可以將計算後的數據2返回到main()裡，main()裡也可以直接向subgen()拋入異常以終止subgen ()。

## 利用 yield from 重構

抽象socket連接的功能:

In [38]:
import socket
from selectors import DefaultSelector, EVENT_WRITE, EVENT_READ

selector = DefaultSelector()
stopped = False
urls_todo = {'/', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9'}


def connect(sock, address):
    f = Future()
    sock.setblocking(False)
    try:
        sock.connect(address)
    except BlockingIOError:
        pass

    def on_connected():
        f.set_result(None)

    selector.register(sock.fileno(), EVENT_WRITE, on_connected)
    yield from f
    selector.unregister(sock.fileno())

抽象單次recv()和讀取完整的response功能:

In [39]:
def read(sock):
    f = Future()

    def on_readable():
        f.set_result(sock.recv(4096))

    selector.register(sock.fileno(), EVENT_READ, on_readable)
    chunk = yield from f
    selector.unregister(sock.fileno())
    return chunk


def read_all(sock):
    response = []
    chunk = yield from read(sock)
    while chunk:
        response.append(chunk)
        chunk = yield from read(sock)
    return b''.join(response)

三個關鍵點的抽像已經完成，現在重構Crawler類

In [40]:
class Crawler:
    def __init__(self, url):
        self.url = url
        self.response = b''

    def fetch(self):
        global stopped
        sock = socket.socket()
        yield from connect(sock, ('example.com', 80))
        get = 'GET {0} HTTP/1.0\r\nHost: example.com\r\n\r\n'.format(self.url)
        sock.send(get.encode('ascii'))
        self.response = yield from read_all(sock)
        urls_todo.remove(self.url)
        if not urls_todo:
            stopped = True

上面代碼整體來講沒什麼問題，可複用的代碼已經抽像出去，作為子生成器也可以使用 yield from 語法來獲取值。但另外有個點需要注意：在第24和第35行返回future對象的時候，我們了yield from f 而不是原來的yield f。 yield可以直接作用於普通Python對象，而yield from卻不行，所以我們對Future還要進一步改造，把它變成一個iterable對象就可以了。

In [41]:
class Future:
    def __init__(self):
        self.result = None
        self._callbacks = []

    def add_done_callback(self, fn):
        self._callbacks.append(fn)

    def set_result(self, result):
        self.result = result
        for fn in self._callbacks:
            fn(self)

    def __iter__(self):
        yield self
        return self.result

只是增加了__iter__()方法的實現。如果不把Future改成iterable也是可以的，還是用原來的yield f即可。那為什麼需要改進呢？

首先，我們是在基於生成器做協程，而生成器還得是生成器，如果繼續混用yield和yield from 做協程，代碼可讀性和可理解性都不好。其次，如果不改，協程內還得關心它等待的對像是否可被yield，如果協程裡還想繼續返回協程怎麼辦？如果想調用普通函數動態生成一個Future對象再返回怎麼辦？

所以，在Python 3.3 引入yield from新語法之後，就不再推薦用yield去做協程。全都使用yield from由於其雙向通道的功能，可以讓我們在協程間隨心所欲地傳遞數據。

In [42]:
# 把 task 跟 event loop 加進來跑跑看

class Task:
    def __init__(self, coro):
        self.coro = coro
        f = Future()
        f.set_result(None)
        self.step(f)

    def step(self, future):
        try:
            # send会进入到coro执行, 即fetch, 直到下次yield
            # next_future 为yield返回的对象
            next_future = self.coro.send(future.result)
        except StopIteration:
            return
        next_future.add_done_callback(self.step)


def loop():
    while not stopped:
        # 阻塞, 直到一个事件发生
        events = selector.select()
        for event_key, event_mask in events:
            callback = event_key.data
            callback()


if __name__ == '__main__':
    import time
    start = time.time()
    for url in urls_todo:
        crawler = Crawler(url)
        Task(crawler.fetch())
    loop()
    print(time.time() - start)


0.5054280757904053


## yield from改進協程總結

用yield from改進基於生成器的協程，代碼抽象程度更高。使業務邏輯相關的代碼更精簡。由於其雙向通道功能可以讓協程之間隨心所欲傳遞數據，使Python異步編程的協程解決方案大大向前邁進了一步。

於是Python語言開發者們充分利用yield from，使 Guido 主導的Python異步編程框架Tulip迅速脫胎換骨，並迫不及待得讓它在 Python 3.4 中換了個名字asyncio以“實習生”角色出現在標準庫中。

## asyncio 介紹
asyncio是Python 3.4 試驗性引入的異步I/O框架（PEP 3156），提供了基於協程做異步I/O編寫單線程並發代碼的基礎設施。其核心組件有事件循環（Event Loop）、協程(Coroutine）、任務(Task)、未來對象(Future)以及其他一些擴充和輔助性質的模塊。

在引入asyncio的時候，還提供了一個裝飾器@asyncio.coroutine用於裝飾使用了yield from的函數，以標記其為協程。但並不強制使用這個裝飾器。

雖然發展到Python 3.4 時有了yield from的加持讓協程更容易了，但是由於協程在Python中發展的歷史包袱所致，很多人仍然弄不明白生成器和協程的聯繫與區別，也弄不明白yield 和yield from 的區別。這種混亂的狀態也違背Python之禪的一些準則。

於是Python設計者們又快馬加鞭地在 3.5 中新增了async/await語法（PEP 492），對協程有了明確而顯式的支持，稱之為原生協程。 async/await 和 yield from這兩種風格的協程底層復用共同的實現，而且相互兼容。

在Python 3.6 中asyncio庫“轉正”，不再是實驗性質的，成為標準庫的正式一員。

## 總結

行至此處，我們已經掌握了asyncio的核心原理，學習了它的原型，也學習了異步I/O在 CPython 官方支持的生態下是如何一步步發展至今的。

實際上，真正的asyncio比我們前幾節中學到的要復雜得多，它還實現了零拷貝、公平調度、異常處理、任務狀態管理等等使 Python 異步編程更完善的內容。理解原理和原型對我們後續學習有莫大的幫助。

## 原生 asyncio 改寫


In [43]:
import asyncio
import aiohttp

host = 'http://example.com'
urls_todo = {'/', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9'}

loop = asyncio.get_event_loop()


async def fetch(url):
    async with aiohttp.ClientSession(loop=loop) as session:
        async with session.get(url) as response:
            response = await response.read()
            return response


if __name__ == '__main__':
    import time
    start = time.time()
    tasks = [fetch(host + url) for url in urls_todo]
    loop.run_until_complete(asyncio.gather(*tasks))
    print(time.time() - start)

0.5892322063446045


對比生成器版的協程，使用asyncio庫後變化很大：

沒有了yield 或 yield from，而是async/await
沒有了自造的loop()，取而代之的是asyncio.get_event_loop()
無需自己在socket上做異步操作，不用顯式地註冊和註銷事件，aiohttp庫已經代勞
沒有了顯式的 Future 和 Task，asyncio已封裝
更少量的代碼，更優雅的設計
說明：我們這裡發送和接收HTTP請求不再自己操作socket的原因是，在實際做業務項目的過程中，要處理妥善地HTTP協議會很複雜，我們需要的是功能完善的異步HTTP客戶端，業界已經有了成熟的解決方案，DRY不是嗎？

和同步阻塞版的代碼對比：

異步化
代碼量相當（引入aiohttp框架後更少）
代碼邏輯同樣簡單，跟同步代碼一樣的結構、一樣的邏輯
接近10倍的性能提升
